In [26]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler
from scipy.stats import ks_2samp


In [27]:
#Cargar el modelo y el preprocesador
modelo = joblib.load("model_heuristic.pkl")
preprocesador = joblib.load("preprocessor.pkl")

print("Modelo y preprocesador cargados correctamente.")


Modelo y preprocesador cargados correctamente.


In [28]:
data = pd.read_csv("telecom_churn.csv")

# Variable objetivo CHURN
X_original = data.drop("Churn", axis=1)
y_original = data["Churn"]

print("Dataset original cargado con", X_original.shape[0], "registros.")

Dataset original cargado con 3333 registros.


In [29]:
# Simulación con nuevos datos
X_nuevo = X_original.sample(300, random_state=42).copy()

# Simular drift alterando algunos valores
X_nuevo["AccountWeeks"] = X_nuevo["AccountWeeks"] * np.random.uniform(0.9, 1.1, size=len(X_nuevo))
X_nuevo["DayMins"] = X_nuevo["DayMins"] * np.random.uniform(0.85, 1.15, size=len(X_nuevo))

print("Datos nuevos simulados correctamente.")

Datos nuevos simulados correctamente.


In [30]:
import numpy as np
import pandas as pd
from scipy.stats import ks_2samp

# Comparación de las estadísticas entre el dataset original y el nuevo
def comparar_distribuciones(df_old, df_new):
    print("\nComparación de distribuciones (prueba KS):")
    
    resultados_drift = []

    for col in df_old.columns:
        if np.issubdtype(df_old[col].dtype, np.number):
            ks_stat, p_value = ks_2samp(df_old[col], df_new[col])
            drift = "Posible drift" if p_value < 0.05 else "Estable"
            resultados_drift.append([col, round(ks_stat, 4), round(p_value, 4), drift])

    tabla_drift = pd.DataFrame(resultados_drift, columns=["Variable", "KS-Stat", "p-value", "Conclusión"])
    return tabla_drift


# Simulación de datos (puedes cambiar por tus datasets reales)
df_original = pd.read_csv("telecom_churn.csv")
df_nuevo = df_original.copy()
df_nuevo["MonthlyCharge"] = df_nuevo["MonthlyCharge"] * 1.1  # simula cambio

# Ejecutar comparación
tabla_drift = comparar_distribuciones(df_original, df_nuevo)
display(tabla_drift)



Comparación de distribuciones (prueba KS):


,Variable,KS-Stat,p-value,Conclusión
0,Churn,0.0000,1.0,Estable
1,AccountWeeks,0.0000,1.0,Estable
2,ContractRenewal,0.0000,1.0,Estable
3,DataPlan,0.0000,1.0,Estable
4,DataUsage,0.0000,1.0,Estable
5,CustServCalls,0.0000,1.0,Estable
6,DayMins,0.0000,1.0,Estable
7,DayCalls,0.0000,1.0,Estable
8,MonthlyCharge,0.1632,0.0,Posible drift
9,OverageFee,0.0000,1.0,Estable


In [35]:
# Indicador visual de riesgo
def color_alerta(p):
    if p < 0.01:
        return "🔴 Alto"
    elif p < 0.05:
        return "🟠 Medio"
    else:
        return "🟢 Bajo"

# Aplicar el semáforo a la tabla de resultados
tabla_drift["Nivel de riesgo"] = tabla_drift["p-value"].apply(color_alerta)
display(tabla_drift)

# Detección automática de drift significativo
drift_detectado = tabla_drift[tabla_drift["p-value"] < 0.05]

if not drift_detectado.empty:
    print("Se detectaron desviaciones significativas en las siguientes variables:")
    display(drift_detectado[["Variable", "p-value", "Conclusión", "Nivel de riesgo"]])
else:
    print("No se detectaron desviaciones significativas. El modelo está estable.")



,Variable,KS-Stat,p-value,Conclusión,Nivel de riesgo
0,Churn,0.0000,1.0,Estable,🟢 Bajo
1,AccountWeeks,0.0000,1.0,Estable,🟢 Bajo
2,ContractRenewal,0.0000,1.0,Estable,🟢 Bajo
3,DataPlan,0.0000,1.0,Estable,🟢 Bajo
4,DataUsage,0.0000,1.0,Estable,🟢 Bajo
5,CustServCalls,0.0000,1.0,Estable,🟢 Bajo
6,DayMins,0.0000,1.0,Estable,🟢 Bajo
7,DayCalls,0.0000,1.0,Estable,🟢 Bajo
8,MonthlyCharge,0.1632,0.0,Posible drift,🔴 Alto
9,OverageFee,0.0000,1.0,Estable,🟢 Bajo


Se detectaron desviaciones significativas en las siguientes variables:


,Variable,p-value,Conclusión,Nivel de riesgo
8,MonthlyCharge,0.0,Posible drift,🔴 Alto


In [41]:
# Detección automática de drift significativo (versión corregida)
drift_detectado = tabla_drift[tabla_drift["p-value"] < 0.05]

if not drift_detectado.empty:
    print("Se detectaron desviaciones significativas en las siguientes variables:")
    display(drift_detectado[["Variable", "p-value", "Conclusión", "Nivel de riesgo"]])
else:
    print("No se detectaron desviaciones significativas. El modelo está estable.")


Se detectaron desviaciones significativas en las siguientes variables:


,Variable,p-value,Conclusión,Nivel de riesgo
8,MonthlyCharge,0.0,Posible drift,🔴 Alto


In [ ]:
_dntk.DeepnoteChart(tabla_drift, """{"mark":{"type":"bar","tooltip":true},"width":"container","config":{"customFormatTypes":true},"height":"container","$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{"x":{"sort":null,"type":"nominal","scale":{"type":"linear"},"format":{"type":"default","decimals":null}},"y":{"sort":null,"type":"nominal","scale":{"type":"linear"},"format":{"type":"default","decimals":null}}},"usermeta":{"tooltipDefaultMode":true}}""", attach_selection=True, filters='[]')

In [ ]:
_dntk.DeepnoteChart(tabla_drift, """{"mark":{"type":"bar","tooltip":true},"width":"container","config":{"customFormatTypes":true},"height":"container","$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{"x":{"sort":null,"type":"nominal","scale":{"type":"linear"},"format":{"type":"default","decimals":null}},"y":{"sort":null,"type":"nominal","scale":{"type":"linear"},"format":{"type":"default","decimals":null}}},"usermeta":{"tooltipDefaultMode":true}}""", attach_selection=True, filters='[]')

Insights:

-Todas las variables que se analizaron presentan una estabilidad estadística buena, ya que los valores de p-value son mayores a 0.05.

-No hubieron cambios significativos a la hora de distribuir los datos con respecto a los del entrenamiento original.

-Algunas variables como Accountweeks, MonthlyCharge, RoaMins y DayMins se mantienen con comportamientos iguales, lo cual indica que el entorno de negocio no se ha cambiado de una manera abrupta.

-No se detectaron desviaciones significativas, por lo que este modelo conserva su precisión. 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4f703592-4945-45ea-93be-011448982d02' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>